In [1]:
%%writefile src/MyBank.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract MyBank {
    address owner; //address payable owner;
    uint balance;
    constructor() { //constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function withdraw(uint amount) public payable {
        balance -= amount;   // 송금 전 감액
        payable(owner).transfer(amount); // 이전 owner.transfer(amount)
    }
    function transferTo(address payable receiver, uint amount) public payable {
        balance -= amount;   // 송금 전 감액
        receiver.transfer(amount);
    }
    function getBalance() public view returns (uint) {
        return balance;
    }
    function getBalanceOfThis() public view returns (uint) {
        return address(this).balance;
    }
    function getBalanceOfOwner() public view returns (uint) {
        return owner.balance;
    }
}

Overwriting src/MyBank.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/MyBank.sol > src/MyBank.json

In [3]:
!type src\MyBank.json

{"contracts":{"src/MyBank.sol:MyBank":{"abi":[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceOfOwner","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalanceOfThis","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address payable","name":"receiver","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"na

In [4]:
%%writefile src/MyBankDeployAbiBinFromFile.js
var Web3=require('web3');
var web3=new Web3('http://localhost:8345');

var _abiBinJson = require('./MyBank.json');
contractName = Object.keys(_abiBinJson.contracts);
_abi = _abiBinJson.contracts[contractName].abi;
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying teh contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: '0x'+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash);
        })
    console.log("---> The contract deployed to: " + deployed.options.address);
}
deploy()

Overwriting src/MyBankDeployAbiBinFromFile.js


In [5]:
!node src/MyBankDeployAbiBinFromFile.js

Deploying teh contract from 0xA3Df2B486ECf9A0D21d503C200dE78D878237954
hash: 0x81ad2fdf7b5bfef9e274c574dacf40bcaa295d979d6f1f47c1bcd4e9de36c84f
---> The contract deployed to: 0x80Bde13dd2C82aC90E29412aca13CFaC5D4f50a0


In [6]:
%%writefile src/MyBankUse.js
var Web3=require('web3');
var web3=new Web3('http://localhost:8345');

var _abiBinJson = require('./MyBank.json');
contractName = Object.keys(_abiBinJson.contracts);
abi = _abiBinJson.contracts[contractName].abi;

var myBank = new web3.eth.Contract(abi, "0x80Bde13dd2C82aC90E29412aca13CFaC5D4f50a0");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    //console.log("Call from: " + accounts[0]);

    await myBank.methods.deposit(1122).send({from: accounts[0], value:1122});
    await myBank.methods.getBalance().call().then(console.log);
    await myBank.methods.withdraw(1111).send({from: accounts[0]});
    await myBank.methods.getBalance().call().then(console.log);
    await myBank.methods.transferTo(accounts[3], 11).send({from: accounts[0]});
    await myBank.methods.getBalance().call().then(console.log);
    bal3 = await web3.eth.getBalance(accounts[3])
    console.log("account3 balance: ", bal3);
}

doIt();

Overwriting src/MyBankUse.js


In [7]:
!node src/MyBankUse.js

1122
11
0
account3 balance:  1000000000000000000011


In [30]:
# 11주 강의자료 MyBank.sol을 ganache@8345에 배포하고, 다음을 테스트하세요.

# - 은행에 1122 입금하고, 잔고 출력

# - 은행에서 1111 출금하고, 잔고 출력

# - transferTo() 함수를 호출해서, 자신의 account3으로 이체하고, 잔고 출력